In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
os.chdir("..")
print(os.getcwd())
sys.path.insert(0, os.getcwd())

/home/grendelyang/NFGP


In [2]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [3]:
!ls -d logs/*/

logs/armidillo_ballet_s1e-1_b1e-3_2021-Dec-04-19-40-13/
logs/checkpoints/
logs/images/
logs/jolteon_nosedown_legsapart_s1e-1_b1e-3_2021-Dec-05-10-17-37/
logs/jolteon_nosedown_legsapart_s1e-1_b1e-3_2021-Dec-05-10-18-13/
logs/jolteon_nosedown_legsapart_s1e-1_b1e-3_2021-Nov-30-12-28-56/
logs/jolteon_nosedown_s1e-1_b1e-3_1block_2021-Dec-05-15-27-30/
logs/jolteon_nosedown_s1e-1_b1e-3_2021-Nov-29-16-50-38/
logs/val/
logs/vis/


In [4]:
log_path = "logs/jolteon_nosedown_s1e-1_b1e-3_2021-Nov-29-16-50-38/"
# log_path = "logs/dino_bend_s1e-1_b1e-3_2021-Nov-26-19-16-32/"


In [5]:
import os.path as osp
from utils import load_imf

net, cfg = load_imf(
    log_path, 
    return_cfg=True)
net, cfg

logs/jolteon_nosedown_s1e-1_b1e-3_2021-Nov-29-16-50-38/checkpoints/epoch_139_iters_14000.pt
Original Decoder:
Net(
  (blocks): ModuleList(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
  (act): Sine()
)
Net(
  (blocks): ModuleList(
    (0): InvertibleResBlockLinear(
      (pos_enc): LipBoundedPosEnc()
      (blocks): ModuleList(
        (0): Linear(in_features=33, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=256, bias=True)
        (2): Linear(in_features=256, out_features=3, bias=True)
      )
      (act): ELU(alpha=1.0)
    )
    (1): InvertibleResBlockLinear(
      (po

100%|██████████| 1678/1678 [00:06<00:00, 255.61it/s]


221.62137 40.37848


(DeformationWrapper(
   (orig): Net(
     (blocks): ModuleList(
       (0): Linear(in_features=3, out_features=512, bias=True)
       (1): Linear(in_features=512, out_features=512, bias=True)
       (2): Linear(in_features=512, out_features=512, bias=True)
       (3): Linear(in_features=512, out_features=512, bias=True)
       (4): Linear(in_features=512, out_features=512, bias=True)
       (5): Linear(in_features=512, out_features=512, bias=True)
       (6): Linear(in_features=512, out_features=1, bias=True)
     )
     (act): Sine()
   )
   (deform): Net(
     (blocks): ModuleList(
       (0): InvertibleResBlockLinear(
         (pos_enc): LipBoundedPosEnc()
         (blocks): ModuleList(
           (0): Linear(in_features=33, out_features=256, bias=True)
           (1): Linear(in_features=256, out_features=256, bias=True)
           (2): Linear(in_features=256, out_features=3, bias=True)
         )
         (act): ELU(alpha=1.0)
       )
       (1): InvertibleResBlockLinear(
        

# Load dataset

In [6]:
import trimesh
import importlib
data_lib = importlib.import_module(cfg.data.type)
loader = data_lib.get_data_loaders(cfg.data, None)['train_loader']
for data in loader:
    break
    
print(data.keys())
handles = data['handles'].view(-1, 3).detach().cpu().numpy()
targets = data['targets'].view(-1, 3).detach().cpu().numpy()
print(handles.shape, targets.shape)


p = mp.plot(data['gtr_verts'].cpu().view(-1, 3).numpy(), 
        data['gtr_faces'].cpu().view(-1, 3).numpy())
p.add_points(handles, shading={'point_color': 'red', 'point_size': 0.05})
p.add_points(targets, shading={'point_color': 'blue', 'point_size': 0.05})

dict_keys(['idx', 'handles', 'targets', 'gtr_verts', 'gtr_faces'])
(1289, 3) (1289, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0003545…

2

# Get the output Mesh

In [7]:
from trainers.utils.vis_utils import imf2mesh
res = 256
# out_mesh = imf2mesh(net, res=res, normalize=True, bound=1.)
out_mesh = imf2mesh(net.orig, res=res, normalize=True, bound=1.)

100%|██████████| 1678/1678 [00:06<00:00, 266.84it/s]


221.62137 40.37848


In [8]:
p = mp.plot(out_mesh.vertices.astype(np.float32), out_mesh.faces.astype(np.float32)) 
p.add_points(handles, shading={'point_color': 'red', 'point_size': 0.05})
p.add_points(targets, shading={'point_color': 'blue', 'point_size': 0.05})

/home/grendelyang/anaconda3/envs/NFGP/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/grendelyang/anaconda3/envs/NFGP/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0003809…

2

In [9]:
# Visualize the original neural fields

# Get the intermediate Layers' Output

In [10]:
def forward_n_blocks(net, n):
    def _out_(x):
        ttl = len(net.deform.blocks)
        start = ttl - n
        for i in range(start, ttl):
            x = net.deform.blocks[i](x)
        return net.orig(x)
    return _out_

mesh_lst = []
for i in range(len(net.deform.blocks) + 1):
# for i in [0, len(net.deform.blocks) - 1]:
    print("Nblocks:", i)
    mesh_i = imf2mesh(forward_n_blocks(net, i), res=res, normalize=True, bound=1.)
    print(mesh_i)
    mesh_lst.append(mesh_i)
    
    p = mp.plot(mesh_i.vertices.astype(np.float32), mesh_i.faces.astype(np.float32)) 
    p.add_points(handles, shading={'point_color': 'red', 'point_size': 0.05})
    p.add_points(targets, shading={'point_color': 'blue', 'point_size': 0.05})

Nblocks: 0


100%|██████████| 1678/1678 [00:06<00:00, 263.49it/s]


221.62137 40.37848
<trimesh.Trimesh(vertices.shape=(88989, 3), faces.shape=(177966, 3))>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0003809…

Nblocks: 1


...

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.078841…

Nblocks: 6


100%|██████████| 1678/1678 [00:16<00:00, 99.29it/s] 


211.79465 40.361458
<trimesh.Trimesh(vertices.shape=(88736, 3), faces.shape=(177480, 3))>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0078779…